In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from toolz import memoize

# Final Review: Programming, Tables, Histograms

# Functions
Functions encapsulates code so that we (or a higher-order function like `apply`) can use it over and over again without rewriting it.

Structure of a function definition:

In [5]:
def <function name> (<zero or more arguments>):
    <body>
    return <some value> # optional

SyntaxError: invalid syntax (<ipython-input-5-922ef6a79e62>, line 1)

Above, the body of the function can be reused over and over again whenever we call the function by the function name. 

## Functions and Return Values
All functions have return values, including the ones provided for us to use with `Tables` and `Arrays`. 
* We can write a function without a return value, but that's usually not the case.

Suppose we have the following `population` table,

In [ ]:
year = np.arange(1999, 2003, 1)
total_pop = [500, 638, 752, 806]
population = Table().with_columns(
    'year', year,
    'total_pop', total_pop
)
population

`.where` returns a Table, so we can call `scatter` after.

In [6]:
population.where('year', are.above(1999)).scatter('year')

NameError: name 'population' is not defined

However, if we call `.scatter` first, `scatter` returns `None`. Thus, we can't call `where` after,

In [ ]:
population.scatter('year').where('year', are.above(1999))

We can see that there's a certain order for calling functions when working with arrays and tables. 

## Try Writing a Function
Write a function `hungry_adder` that does the following,

1. Takes 2 numbers as arguments
2. Adds them together and returns the value
3. Prints the string "Thanks for feeding me those tasty number!"

In [ ]:
def hungry_adder(a, b):
    print("Thanks for feeding me those tasty number!")
    return a + b

In [ ]:
hungry_adder(2, 4)

Make sure to write the `print` statement before the `return` statement. Once we get to the `return` statement, the function ends. If we write the `print` statement after the `return` statement,

In [ ]:
def hungry_adder(a, b):
    return a + b
    print("Thanks for feeding me those tasty numbers!")

In [ ]:
hungry_adder(2, 4)

Then the `print` statement is never reached!

# Arrays and Tables

## Arrays
Arrays contain values. They are useful because we can work with multiple values at once. 

To make an array, use `make_array` and put elements in it. 

In [ ]:
make_array(1, 2, 3)

Below are some element-wise array operations,

In [ ]:
2 * make_array(1, 2, 3)

In [ ]:
make_array(2, 3, 5) + make_array(7, 8, 9) # Arrays must be the same length

## Tables
Table is a sequence of labeled columns.
* Labels are `strings`
* Columns are `arrays`

1. `table.column('<label of column>')`
    * Gives back the column of the table as an array
2. `table.select('<label of column>')`
    * Returns a new table with just that one column

In tables, all data in a column need to be the same type. 

<img src = 'table.jpg' width = 500/>

Recall that we can also view table as a collection of rows. 

## Where

In [7]:
tbl.where(column, predicate)

NameError: name 'tbl' is not defined

`.where` returns a table of the rows for which the column satisfies some predicate
* "Filter" out rows by a condition
* Returns a table

List of `where` predicates: [Here](https://www.inferentialthinking.com/chapters/06/2/Selecting_Rows.html)


## Join
The `join` method combines 2 tables. It takes 3 arguments:
1. Column to join on from the first table
2. Second table
3. Column to join on from the second table

## Join Example
Let's say we have the following tables,`sp16` and `prof`,

In [9]:
SID = make_array(1234, 5678, 5678)
Class = make_array('CS 61A', 'CS 61A', 'DATA 8')
GPA = make_array(3.3, 3.0, 4.0)
sp16 = Table().with_column(
    'Student ID', SID,
    'Class', Class,
    'GPA', GPA
)
sp16

Student ID,Class,GPA
1234,CS 61A,3.3
5678,CS 61A,3
5678,DATA 8,4


In [10]:
Course = make_array('CS 61A', 'DATA 8')
Professor = make_array('Hilfinger', 'DeNero')
prof = Table().with_column(
    'Course', Course,
    'Professor', Professor
)
prof

Course,Professor
CS 61A,Hilfinger
DATA 8,DeNero


Above, we see that the `Class` column from `sp16` table and the `Course` column from `prof` table looks similar. We can join these columns! However, beware that the order we join the tables is important!

## Does Order Matter?

In [11]:
prof.join('Course', sp16, 'Class')

Course,Professor,Student ID,GPA
CS 61A,Hilfinger,1234,3.3
CS 61A,Hilfinger,5678,3
DATA 8,DeNero,5678,4


In [12]:
sp16.join('Class', prof, 'Course')

Class,Student ID,GPA,Professor
CS 61A,1234,3.3,Hilfinger
CS 61A,5678,3,Hilfinger
DATA 8,5678,4,DeNero


See that if we change the order, the output table is different! This is because the `.join` method:
1. Takes the 1st table
2. Goes through every single value in the column that has matching rows
3. Finds the matching row from the other table to join on 

## Group
If we found ourselves doing `.where` multiple times on the same column, chances are we want to do `group` instead. `.group` takes a column, grabs all the unique values in the column and groups them together according to the function that we defined. 

In [13]:
table.group("Column name", function)

NameError: name 'table' is not defined

`.group` group table by categories. It takes 2 arguments:
1. Column we want to group by
    * Split column into unique values

2. (Optional) aggregate function
    * If we don't provide any argument here, then `.group` just generates a new table with one extra column `counts` that counts how many times each unique values appear 

## Group -- Function Argument
We should know the the type of the item we passed in to the function. The function needs to be able to take an array. 

1. No argument: only `count` column
    * Calculates the number of rows for each category


2. Argument: "ColName FunctionName" column
    * Applies function to values in "ColName" for each category
    * `sum`, `mean`, `min`, `max`, `count`, `list`, `abs`, or we can make our own functions and pass them in. 

## Group Example - `sp16` table
We have the following 

In [14]:
sp16

Student ID,Class,GPA
1234,CS 61A,3.3
5678,CS 61A,3
5678,DATA 8,4


What would happen if we run the following,

In [17]:
sp16.drop('Class').group('Student ID', np.mean)

Student ID,GPA mean
1234,3.3
5678,3.5


The cell does the following,

1. Drops the `Class` column
2. `group` by unique `Student ID` and call `np.mean` on the values in each group

## Group with Multiple Columns
Group table by multiple levels of categories
* When we group by 1 column, we look for the unique value in that one particular column. On the other hand, when we group by multiple columns, we look at unique pairs of values in 2 columns or unique triplets of values in 3 columns, and so on.
    
`group` by multiple columns takes 2 arguments:
1. Array of column names
2. A function

If we group by `Column 1` and `Column 2`, we'll get all the unique pairs in both columns and create a group out of each.

Example syntax:

In [18]:
table.group(make_array('ColName1', 'ColName2'), func)

NameError: name 'table' is not defined

## Pivot
If we want to use `group` with multiple columns, but we want the format of the table to look different, chances are we want to use `pivot`. 

Pivot redesigns table structure based on chosen categories. The arguments are as the following (it takes 2-4 arguments):
1. Category unique on the columns
2. Category unique on the rows
    * Note: If we don't provide 3rd and 4th arguments, the default values are counts
3. Values to aggregate
4. Aggregate function

## Pivot Example
Again, we have the table `sp16`,

In [3]:
sp16

NameError: name 'sp16' is not defined

If we pivot `Student ID` and `Class`,

In [ ]:
sp16.pivot('Student ID', 'Class')

By default, `pivot` counts each occurrence of unique combination of the 2 columns `Student ID` and `Class`. As we can see above, there is no row in `sp16` where the `Student ID` is `1234` and `Class` is `DATA 8` on the same time.

If we call the following,

In [ ]:
sp16.pivot('Student ID', 'Class', 'GPA', np.mean)

Then we count the `GPA` of the unique combination of the 2 columns `Student ID` and `Class`. `pivot` didn't calculate `mean` at all since all the combinations are unique and thus, the `mean` of a value is the value itself.

## Practice Problem 1
We have a table `exports` containing the export amounts (in millions of dollars) of various agricultural products from California in the year 2014. The first column is labeled `Product`, while the second is labeled `amount`.

1. Write a line of code that computes the average of the **amount** column
2. Write a line of code that returns a table with only the products that are above average.
3. Write a line of code that returns `True` if any exports are less than 1 million dollars
4. Write a line of code that returns the proportion of exports that are between 50 and 100 million dollars

### Ans:
Assume `exports` is as the following,

In [4]:
Products = make_array('Wool', 'Cotton', 'Kryptonite')
amount = make_array(73, 0.3, 4)
exports = Table().with_columns(
    'Products', Products,
    'amount', amount
)
exports

Products,amount
Wool,73
Cotton,0.3
Kryptonite,4


For the first problem, 

In [5]:
avg = np.average(exports.column('amount'))
avg

25.766666666666666

For the 2nd problem,

In [6]:
exports.where('amount', are.above(avg))

Products,amount
Wool,73


3rd problem is tricky. Since it asks for a value that's less than a certain value, we can sort the table based on increasing `amount`, and check if the first row `amount` is less than `1`

In [7]:
exports.sort('amount', descending = False).column('amount').item(0) < 1

True

For the 4th problem,

In [8]:
exports.where('amount', are.between(50, 100)).num_rows / exports.num_rows

0.3333333333333333

## Practice Problem 2
Alice wants to pick a new TV show to watch, and decides to use some data she found online about various TV shows to make a decision. The table of TV shows she is using is called `tv_shows`, and each row corresponds to a unique show. The first 6 rows of the table are shown above.

### #1
There are initially too many options to choose from, so Alice decides to filter the table to only include shows that satisfy both of the following conditions:
1. Has at least `8.0` rating
2. Has at least `6` seasons

Write an expression that will filter `tv_shows` such that only the rows satisfying both of the above conditions will be included. Below is the `tv_shows` table,

In [9]:
name = make_array("Grey's Anatomy", "Suits", "House of Cards", "Scrubs", "Scandal", "How I Met Your Mother")
rating = make_array(7.7, 8.7, 9, 8.4, 7.9, 8.4)
season = make_array(12, 5, 4, 8, 5, 9)
genre = make_array("medical drama", "legal drama", "political drama", "sitcom", "political drama", "sitcom")
premiere = make_array(2005, 2011, 2013, 2001, 2012, 2005)

tv_shows = Table().with_columns(
    'Name', name,
    'Rating', rating,
    '# of Seasons', season,
    'Genre', genre,
    'Premiere Year', premiere
)
tv_shows

Name,Rating,# of Seasons,Genre,Premiere Year
Grey's Anatomy,7.7,12,medical drama,2005
Suits,8.7,5,legal drama,2011
House of Cards,9,4,political drama,2013
Scrubs,8.4,8,sitcom,2001
Scandal,7.9,5,political drama,2012
How I Met Your Mother,8.4,9,sitcom,2005


In [10]:
(tv_shows
 # Has at least 8.0 rating
 .where('Rating', are.above_or_equal_to(8.0))
 # Has at least 6 seasons
 .where('# of Seasons', are.above_or_equal_to(6))
)

Name,Rating,# of Seasons,Genre,Premiere Year
Scrubs,8.4,8,sitcom,2001
How I Met Your Mother,8.4,9,sitcom,2005


## #2
Alice realizes that she doesn't even know what type of show she wants to watch, so she decides to look at what genres people seem to like best.

First, write an expression that will create a **table with the average rating for each `Genre`** and assign it to the variable `shows_grouped_by_genre`

Then, use this table to **find the Genre with the highest average rating** and assign it to the variable `highest_rated_genre`

In [11]:
avg_rating = tv_shows.select('Genre', 'Rating').group('Genre', np.mean)
avg_rating

Genre,Rating mean
legal drama,8.7
medical drama,7.7
political drama,8.45
sitcom,8.4


In [12]:
highest_rated_genre = avg_rating.sort('Rating mean', descending = True).column('Genre').item(0)
highest_rated_genre

'legal drama'

## #3
In addition to looking at the ratings of TV shows classified by genre, Alice also cares about how old the show is, since she doesn't want to watch a show where she doesn't recognize any of the actors / actresses.

Write an expression to help Alice create a `pivot` table where:
1. The rows correspond to whether or not the show premiered after 2008
2. The columns correspond to the genre of the show
3. The values in the table are the average rating of shows corresponding to each row / column pair

In [16]:
tv_shows.with_column(
    'After 2008', tv_shows.column('Premiere Year') > 2008 
).pivot('Genre', 'After 2008', 'Rating', np.mean)

After 2008,legal drama,medical drama,political drama,sitcom
False,0,7.7,0,8.4
True,8.7,0,8.45,0
